## Import Libraries

In [1]:
!pip3 install pandas scikit-learn --break-system-packages

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB, ComplementNB

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 5.0 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 4.8 MB/s  0:00:02m0:00:0100:01
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 4.5 MB/s  0:00:01 eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 4.6 MB/s  0:00:06 eta 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [scikit-learn] [scikit-learn]

[notice] 

## Load Dataset

In [2]:
df = pd.read_csv("../datasets/ResumeDataSet.csv")
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


## Split into Train & Test

In [3]:
x, y = df["Resume"], df["Category"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.35, random_state=42)

## Create NLP Pipeline Function

In [4]:
def make_model(model):
    return Pipeline([
        ('tfidf', TfidfVectorizer(encoding='utf-8', decode_error='ignore',lowercase=True,
                                  stop_words='english', token_pattern=r'(?u)\b[a-zA-Z]{2,}\b',)),
        ('scale', MaxAbsScaler()),
        ('select', SelectKBest(k=5000)),
        ('model', model)
    ])

## Define Models to Compare

In [5]:
models = {
    "Multinomial NB": MultinomialNB(),
    "Complement NB": ComplementNB(),
}

## Train & Evaluate Models

In [6]:
for name, model in models.items():
    pipe = make_model(model)
    pipe.fit(x_train, y_train)
    preds = pipe.predict(x_test)
    print(f"----- {name} -----")
    print("Accuracy:", accuracy_score(y_test, preds))
    print(classification_report(y_test, preds))

----- Multinomial NB -----
Accuracy: 0.9584569732937686
                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         7
                     Arts       1.00      1.00      1.00        12
       Automation Testing       1.00      1.00      1.00         7
               Blockchain       1.00      1.00      1.00        13
         Business Analyst       1.00      1.00      1.00         9
           Civil Engineer       1.00      0.53      0.70        15
             Data Science       1.00      1.00      1.00        13
                 Database       1.00      1.00      1.00        10
          DevOps Engineer       1.00      0.90      0.95        21
         DotNet Developer       1.00      0.55      0.71        11
            ETL Developer       1.00      1.00      1.00        10
   Electrical Engineering       1.00      1.00      1.00        11
                       HR       1.00      1.00      1.00        19
     